In [4]:
import numpy as np
import pickle
from openeye import oechem
import copy
import mdtraj as md
import os

# Get the water indices (to be turned into ion) for complex

In [70]:
with open("/data/chodera/zhangi/perses_benchmark/neq/15/104/104_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [71]:
# Read in lambda = 0 cache
with open(os.path.join("/data/chodera/zhangi/perses_benchmark/neq/15/104/", f"104_complex_asp_1ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[0]

In [ ]:
positions.shape

In [8]:
# Identify which indices (hybrid topology) are the water atoms that is being turned into an ion

# New topology indices
# Apo: [37532 37533 37534]
# Complex: [150820 150821 150822]

In [72]:
for k, v in htf._hybrid_to_new_map.items():
    if v in [150820, 150821, 150822]:
        print(k)

150811
150812
150813


In [75]:
for atom in htf.hybrid_topology.atoms:
    if atom.index in [150811, 150812, 150813]:
        print(atom)

HOH46139-O
HOH46139-H1
HOH46139-H2


In [73]:
htf.hybrid_topology.n_atoms

183508

# Write function for choosing indices to swap with

In [12]:
def choose_water_atoms(charge_diff, old_indices, hybrid_positions, hybrid_topology, radius=0.8):
    import mdtraj as md
    # Create trajectory
    traj = md.Trajectory(hybrid_positions, hybrid_topology)
    water_atoms = traj.topology.select(f"water")
    query_atoms = traj.top.select('protein')

    while True:
        # Get water atoms within radius of protein
        neighboring_atoms = md.compute_neighbors(traj, radius, query_atoms, haystack_indices=water_atoms)[0]

        # Get water atoms outside of radius of protein
        nonneighboring_residues = set([atom.residue.index for atom in traj.topology.atoms if (atom.index in water_atoms) and (atom.index not in neighboring_atoms)])
        assert len(nonneighboring_residues) > 0, "there are no available nonneighboring waters"
        # Choose N random nonneighboring waters, where N is determined based on the charge_diff
        choice_residues = np.random.choice(list(nonneighboring_residues), size=abs(charge_diff), replace=False)

        # Get the atom indices in the water(s)
        choice_indices = np.array([[atom.index for atom in traj.topology.residue(res).atoms] for res in choice_residues])
        
        if not np.array_equal(choice_indices, np.array(old_indices)):
            break
        
    return np.ndarray.flatten(choice_indices)


In [13]:
traj = md.Trajectory(htf.hybrid_positions, htf.hybrid_topology)
water_atoms = traj.topology.select(f"water")
query_atoms = traj.top.select('protein')

In [76]:
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom)

0 ACE332-H1
1 ACE332-CH3
2 ACE332-H2
3 ACE332-H3
4 ACE332-C
5 ACE332-O
6 THR333-N
7 THR333-H
8 THR333-CA
9 THR333-HA
10 THR333-CB
11 THR333-HB
12 THR333-CG2
13 THR333-HG21
14 THR333-HG22
15 THR333-HG23
16 THR333-OG1
17 THR333-HG1
18 THR333-C
19 THR333-O
20 ASN334-N
21 ASN334-H
22 ASN334-CA
23 ASN334-HA
24 ASN334-CB
25 ASN334-HB2
26 ASN334-HB3
27 ASN334-CG
28 ASN334-OD1
29 ASN334-ND2
30 ASN334-HD21
31 ASN334-HD22
32 ASN334-C
33 ASN334-O
34 LEU335-N
35 LEU335-H
36 LEU335-CA
37 LEU335-HA
38 LEU335-CB
39 LEU335-HB2
40 LEU335-HB3
41 LEU335-CG
42 LEU335-HG
43 LEU335-CD1
44 LEU335-HD11
45 LEU335-HD12
46 LEU335-HD13
47 LEU335-CD2
48 LEU335-HD21
49 LEU335-HD22
50 LEU335-HD23
51 LEU335-C
52 LEU335-O
53 CYS336-N
54 CYS336-H
55 CYS336-CA
56 CYS336-HA
57 CYS336-CB
58 CYS336-HB2
59 CYS336-HB3
60 CYS336-SG
61 CYS336-C
62 CYS336-O
63 PRO337-N
64 PRO337-CD
65 PRO337-HD2
66 PRO337-HD3
67 PRO337-CG
68 PRO337-HG2
69 PRO337-HG3
70 PRO337-CB
71 PRO337-HB2
72 PRO337-HB3
73 PRO337-CA
74 PRO337-HA
75 PRO337-C


1604 TRP436-HE3
1605 TRP436-CD2
1606 TRP436-C
1607 TRP436-O
1608 ASN437-N
1609 ASN437-H
1610 ASN437-CA
1611 ASN437-HA
1612 ASN437-CB
1613 ASN437-HB2
1614 ASN437-HB3
1615 ASN437-CG
1616 ASN437-OD1
1617 ASN437-ND2
1618 ASN437-HD21
1619 ASN437-HD22
1620 ASN437-C
1621 ASN437-O
1622 SER438-N
1623 SER438-H
1624 SER438-CA
1625 SER438-HA
1626 SER438-CB
1627 SER438-HB2
1628 SER438-HB3
1629 SER438-OG
1630 SER438-HG
1631 SER438-C
1632 SER438-O
1633 ASN439-N
1634 ASN439-H
1635 ASN439-CA
1636 ASN439-HA
1637 ASN439-CB
1638 ASN439-HB2
1639 ASN439-HB3
1640 ASN439-CG
1641 ASN439-OD1
1642 ASN439-ND2
1643 ASN439-HD21
1644 ASN439-HD22
1645 ASN439-C
1646 ASN439-O
1647 ASN440-N
1648 ASN440-H
1649 ASN440-CA
1650 ASN440-HA
1651 ASN440-CB
1652 ASN440-HB2
1653 ASN440-HB3
1654 ASN440-CG
1655 ASN440-OD1
1656 ASN440-ND2
1657 ASN440-HD21
1658 ASN440-HD22
1659 ASN440-C
1660 ASN440-O
1661 LEU441-N
1662 LEU441-H
1663 LEU441-CA
1664 LEU441-HA
1665 LEU441-CB
1666 LEU441-HB2
1667 LEU441-HB3
1668 LEU441-CG
1669 LEU441-HG


3339 HOH309-H2
3340 HOH310-O
3341 HOH310-H1
3342 HOH310-H2
3343 HOH311-O
3344 HOH311-H1
3345 HOH311-H2
3346 HOH312-O
3347 HOH312-H1
3348 HOH312-H2
3349 HOH313-O
3350 HOH313-H1
3351 HOH313-H2
3352 HOH314-O
3353 HOH314-H1
3354 HOH314-H2
3355 HOH315-O
3356 HOH315-H1
3357 HOH315-H2
3358 HOH316-O
3359 HOH316-H1
3360 HOH316-H2
3361 HOH317-O
3362 HOH317-H1
3363 HOH317-H2
3364 HOH318-O
3365 HOH318-H1
3366 HOH318-H2
3367 HOH319-O
3368 HOH319-H1
3369 HOH319-H2
3370 HOH320-O
3371 HOH320-H1
3372 HOH320-H2
3373 HOH321-O
3374 HOH321-H1
3375 HOH321-H2
3376 HOH322-O
3377 HOH322-H1
3378 HOH322-H2
3379 HOH323-O
3380 HOH323-H1
3381 HOH323-H2
3382 HOH324-O
3383 HOH324-H1
3384 HOH324-H2
3385 HOH325-O
3386 HOH325-H1
3387 HOH325-H2
3388 HOH326-O
3389 HOH326-H1
3390 HOH326-H2
3391 HOH327-O
3392 HOH327-H1
3393 HOH327-H2
3394 HOH328-O
3395 HOH328-H1
3396 HOH328-H2
3397 HOH329-O
3398 HOH329-H1
3399 HOH329-H2
3400 HOH330-O
3401 HOH330-H1
3402 HOH330-H2
3403 HOH331-O
3404 HOH331-H1
3405 HOH331-H2
3406 HOH332-O
340

5089 HOH893-O
5090 HOH893-H1
5091 HOH893-H2
5092 HOH894-O
5093 HOH894-H1
5094 HOH894-H2
5095 HOH895-O
5096 HOH895-H1
5097 HOH895-H2
5098 HOH896-O
5099 HOH896-H1
5100 HOH896-H2
5101 HOH897-O
5102 HOH897-H1
5103 HOH897-H2
5104 HOH898-O
5105 HOH898-H1
5106 HOH898-H2
5107 HOH899-O
5108 HOH899-H1
5109 HOH899-H2
5110 HOH900-O
5111 HOH900-H1
5112 HOH900-H2
5113 HOH901-O
5114 HOH901-H1
5115 HOH901-H2
5116 HOH902-O
5117 HOH902-H1
5118 HOH902-H2
5119 HOH903-O
5120 HOH903-H1
5121 HOH903-H2
5122 HOH904-O
5123 HOH904-H1
5124 HOH904-H2
5125 HOH905-O
5126 HOH905-H1
5127 HOH905-H2
5128 HOH906-O
5129 HOH906-H1
5130 HOH906-H2
5131 HOH907-O
5132 HOH907-H1
5133 HOH907-H2
5134 HOH908-O
5135 HOH908-H1
5136 HOH908-H2
5137 HOH909-O
5138 HOH909-H1
5139 HOH909-H2
5140 HOH910-O
5141 HOH910-H1
5142 HOH910-H2
5143 HOH911-O
5144 HOH911-H1
5145 HOH911-H2
5146 HOH912-O
5147 HOH912-H1
5148 HOH912-H2
5149 HOH913-O
5150 HOH913-H1
5151 HOH913-H2
5152 HOH914-O
5153 HOH914-H1
5154 HOH914-H2
5155 HOH915-O
5156 HOH915-H1
515

6838 HOH1476-O
6839 HOH1476-H1
6840 HOH1476-H2
6841 HOH1477-O
6842 HOH1477-H1
6843 HOH1477-H2
6844 HOH1478-O
6845 HOH1478-H1
6846 HOH1478-H2
6847 HOH1479-O
6848 HOH1479-H1
6849 HOH1479-H2
6850 HOH1480-O
6851 HOH1480-H1
6852 HOH1480-H2
6853 HOH1481-O
6854 HOH1481-H1
6855 HOH1481-H2
6856 HOH1482-O
6857 HOH1482-H1
6858 HOH1482-H2
6859 HOH1483-O
6860 HOH1483-H1
6861 HOH1483-H2
6862 HOH1484-O
6863 HOH1484-H1
6864 HOH1484-H2
6865 HOH1485-O
6866 HOH1485-H1
6867 HOH1485-H2
6868 HOH1486-O
6869 HOH1486-H1
6870 HOH1486-H2
6871 HOH1487-O
6872 HOH1487-H1
6873 HOH1487-H2
6874 HOH1488-O
6875 HOH1488-H1
6876 HOH1488-H2
6877 HOH1489-O
6878 HOH1489-H1
6879 HOH1489-H2
6880 HOH1490-O
6881 HOH1490-H1
6882 HOH1490-H2
6883 HOH1491-O
6884 HOH1491-H1
6885 HOH1491-H2
6886 HOH1492-O
6887 HOH1492-H1
6888 HOH1492-H2
6889 HOH1493-O
6890 HOH1493-H1
6891 HOH1493-H2
6892 HOH1494-O
6893 HOH1494-H1
6894 HOH1494-H2
6895 HOH1495-O
6896 HOH1495-H1
6897 HOH1495-H2
6898 HOH1496-O
6899 HOH1496-H1
6900 HOH1496-H2
6901 HOH1497-

...

37084 HOH11558-O
37085 HOH11558-H1
37086 HOH11558-H2
37087 HOH11559-O
37088 HOH11559-H1
37089 HOH11559-H2
37090 HOH11560-O
37091 HOH11560-H1
37092 HOH11560-H2
37093 HOH11561-O
37094 HOH11561-H1
37095 HOH11561-H2
37096 HOH11562-O
37097 HOH11562-H1
37098 HOH11562-H2
37099 HOH11563-O
37100 HOH11563-H1
37101 HOH11563-H2
37102 HOH11564-O
37103 HOH11564-H1
37104 HOH11564-H2
37105 HOH11565-O
37106 HOH11565-H1
37107 HOH11565-H2
37108 HOH11566-O
37109 HOH11566-H1
37110 HOH11566-H2
37111 HOH11567-O
37112 HOH11567-H1
37113 HOH11567-H2
37114 HOH11568-O
37115 HOH11568-H1
37116 HOH11568-H2
37117 HOH11569-O
37118 HOH11569-H1
37119 HOH11569-H2
37120 HOH11570-O
37121 HOH11570-H1
37122 HOH11570-H2
37123 HOH11571-O
37124 HOH11571-H1
37125 HOH11571-H2
37126 HOH11572-O
37127 HOH11572-H1
37128 HOH11572-H2
37129 HOH11573-O
37130 HOH11573-H1
37131 HOH11573-H2
37132 HOH11574-O
37133 HOH11574-H1
37134 HOH11574-H2
37135 HOH11575-O
37136 HOH11575-H1
37137 HOH11575-H2
37138 HOH11576-O
37139 HOH11576-H1
37140 HOH11

38834 HOH12141-H1
38835 HOH12141-H2
38836 HOH12142-O
38837 HOH12142-H1
38838 HOH12142-H2
38839 HOH12143-O
38840 HOH12143-H1
38841 HOH12143-H2
38842 HOH12144-O
38843 HOH12144-H1
38844 HOH12144-H2
38845 HOH12145-O
38846 HOH12145-H1
38847 HOH12145-H2
38848 HOH12146-O
38849 HOH12146-H1
38850 HOH12146-H2
38851 HOH12147-O
38852 HOH12147-H1
38853 HOH12147-H2
38854 HOH12148-O
38855 HOH12148-H1
38856 HOH12148-H2
38857 HOH12149-O
38858 HOH12149-H1
38859 HOH12149-H2
38860 HOH12150-O
38861 HOH12150-H1
38862 HOH12150-H2
38863 HOH12151-O
38864 HOH12151-H1
38865 HOH12151-H2
38866 HOH12152-O
38867 HOH12152-H1
38868 HOH12152-H2
38869 HOH12153-O
38870 HOH12153-H1
38871 HOH12153-H2
38872 HOH12154-O
38873 HOH12154-H1
38874 HOH12154-H2
38875 HOH12155-O
38876 HOH12155-H1
38877 HOH12155-H2
38878 HOH12156-O
38879 HOH12156-H1
38880 HOH12156-H2
38881 HOH12157-O
38882 HOH12157-H1
38883 HOH12157-H2
38884 HOH12158-O
38885 HOH12158-H1
38886 HOH12158-H2
38887 HOH12159-O
38888 HOH12159-H1
38889 HOH12159-H2
38890 HOH1

40833 HOH12807-H2
40834 HOH12808-O
40835 HOH12808-H1
40836 HOH12808-H2
40837 HOH12809-O
40838 HOH12809-H1
40839 HOH12809-H2
40840 HOH12810-O
40841 HOH12810-H1
40842 HOH12810-H2
40843 HOH12811-O
40844 HOH12811-H1
40845 HOH12811-H2
40846 HOH12812-O
40847 HOH12812-H1
40848 HOH12812-H2
40849 HOH12813-O
40850 HOH12813-H1
40851 HOH12813-H2
40852 HOH12814-O
40853 HOH12814-H1
40854 HOH12814-H2
40855 HOH12815-O
40856 HOH12815-H1
40857 HOH12815-H2
40858 HOH12816-O
40859 HOH12816-H1
40860 HOH12816-H2
40861 HOH12817-O
40862 HOH12817-H1
40863 HOH12817-H2
40864 HOH12818-O
40865 HOH12818-H1
40866 HOH12818-H2
40867 HOH12819-O
40868 HOH12819-H1
40869 HOH12819-H2
40870 HOH12820-O
40871 HOH12820-H1
40872 HOH12820-H2
40873 HOH12821-O
40874 HOH12821-H1
40875 HOH12821-H2
40876 HOH12822-O
40877 HOH12822-H1
40878 HOH12822-H2
40879 HOH12823-O
40880 HOH12823-H1
40881 HOH12823-H2
40882 HOH12824-O
40883 HOH12824-H1
40884 HOH12824-H2
40885 HOH12825-O
40886 HOH12825-H1
40887 HOH12825-H2
40888 HOH12826-O
40889 HOH12

42833 HOH13474-H1
42834 HOH13474-H2
42835 HOH13475-O
42836 HOH13475-H1
42837 HOH13475-H2
42838 HOH13476-O
42839 HOH13476-H1
42840 HOH13476-H2
42841 HOH13477-O
42842 HOH13477-H1
42843 HOH13477-H2
42844 HOH13478-O
42845 HOH13478-H1
42846 HOH13478-H2
42847 HOH13479-O
42848 HOH13479-H1
42849 HOH13479-H2
42850 HOH13480-O
42851 HOH13480-H1
42852 HOH13480-H2
42853 HOH13481-O
42854 HOH13481-H1
42855 HOH13481-H2
42856 HOH13482-O
42857 HOH13482-H1
42858 HOH13482-H2
42859 HOH13483-O
42860 HOH13483-H1
42861 HOH13483-H2
42862 HOH13484-O
42863 HOH13484-H1
42864 HOH13484-H2
42865 HOH13485-O
42866 HOH13485-H1
42867 HOH13485-H2
42868 HOH13486-O
42869 HOH13486-H1
42870 HOH13486-H2
42871 HOH13487-O
42872 HOH13487-H1
42873 HOH13487-H2
42874 HOH13488-O
42875 HOH13488-H1
42876 HOH13488-H2
42877 HOH13489-O
42878 HOH13489-H1
42879 HOH13489-H2
42880 HOH13490-O
42881 HOH13490-H1
42882 HOH13490-H2
42883 HOH13491-O
42884 HOH13491-H1
42885 HOH13491-H2
42886 HOH13492-O
42887 HOH13492-H1
42888 HOH13492-H2
42889 HOH1

44583 HOH14057-H2
44584 HOH14058-O
44585 HOH14058-H1
44586 HOH14058-H2
44587 HOH14059-O
44588 HOH14059-H1
44589 HOH14059-H2
44590 HOH14060-O
44591 HOH14060-H1
44592 HOH14060-H2
44593 HOH14061-O
44594 HOH14061-H1
44595 HOH14061-H2
44596 HOH14062-O
44597 HOH14062-H1
44598 HOH14062-H2
44599 HOH14063-O
44600 HOH14063-H1
44601 HOH14063-H2
44602 HOH14064-O
44603 HOH14064-H1
44604 HOH14064-H2
44605 HOH14065-O
44606 HOH14065-H1
44607 HOH14065-H2
44608 HOH14066-O
44609 HOH14066-H1
44610 HOH14066-H2
44611 HOH14067-O
44612 HOH14067-H1
44613 HOH14067-H2
44614 HOH14068-O
44615 HOH14068-H1
44616 HOH14068-H2
44617 HOH14069-O
44618 HOH14069-H1
44619 HOH14069-H2
44620 HOH14070-O
44621 HOH14070-H1
44622 HOH14070-H2
44623 HOH14071-O
44624 HOH14071-H1
44625 HOH14071-H2
44626 HOH14072-O
44627 HOH14072-H1
44628 HOH14072-H2
44629 HOH14073-O
44630 HOH14073-H1
44631 HOH14073-H2
44632 HOH14074-O
44633 HOH14074-H1
44634 HOH14074-H2
44635 HOH14075-O
44636 HOH14075-H1
44637 HOH14075-H2
44638 HOH14076-O
44639 HOH14

In [74]:
for atom in query_atoms:
    print(atom)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [34]:
swap_indices = choose_water_atoms(-1, [150813, 150814, 150815], htf.hybrid_positions, htf.hybrid_topology)

In [47]:
swap_indices

array([97175, 97176, 97177])

In [61]:
for atom in htf.hybrid_topology.atoms:
    if atom.index in swap_indices:
        print(atom)

HOH28259-O
HOH28259-H1
HOH28259-H2


In [38]:
og_indices = [150813, 150814, 150815]

In [57]:
for og, swap in zip(og_indices, swap_indices):
    print(positions[og])
    print(positions[swap])
    temp = copy.copy(positions[og])
    positions[og] = positions[swap]
    print(temp)
    positions[swap] = temp

[2.75746489 5.9123044  4.71677208]
[11.61960793 12.97693539  6.68765688]
[2.75746489 5.9123044  4.71677208]
[2.71012211 5.77269173 4.7510767 ]
[11.64937782 12.88596725  6.68673801]
[2.71012211 5.77269173 4.7510767 ]
[2.11051607 5.40358973 4.27983475]
[11.69618893 13.02747536  6.66039324]
[2.11051607 5.40358973 4.27983475]


In [44]:
positions[150813]

array([11.61960793, 12.97693539,  6.68765688])

In [45]:
positions[150814]

array([11.64937782, 12.88596725,  6.68673801])

In [46]:
positions[150815]

array([11.69618893, 13.02747536,  6.66039324])

In [58]:
positions[97175]

array([2.75746489, 5.9123044 , 4.71677208])

In [59]:
positions[97176]

array([2.71012211, 5.77269173, 4.7510767 ])

In [60]:
positions[97177]

array([2.11051607, 5.40358973, 4.27983475])

# Get the water indices (to be turned into ion) for apo

In [41]:
with open("/data/chodera/zhangi/perses_benchmark/neq/15/104/104_apo.pickle", "rb") as f:
    htf = pickle.load(f)

In [59]:
# Read in lambda = 0 cache
with open(os.path.join("/data/chodera/zhangi/perses_benchmark/neq/15/104/", f"104_apo_asp_1ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[0]

In [18]:
# Identify which indices (hybrid topology) are the water atoms that is being turned into an ion

# New topology indices
# Apo: [37532 37533 37534]
# Complex: [150820 150821 150822]

In [43]:
for k, v in htf._hybrid_to_new_map.items():
    if v in [37532, 37533, 37534]:
        print(k)

37523
37524
37525


In [45]:
for atom in htf.hybrid_topology.atoms:
    if atom.index in [37523, 37524, 37525]:
        print(atom)

HOH11705-O
HOH11705-H1
HOH11705-H2


In [47]:
htf.hybrid_topology.n_atoms

44744

In [48]:
swap_indices = choose_water_atoms(-1, [37523, 37524, 37525], htf.hybrid_positions, htf.hybrid_topology)

In [49]:
swap_indices

array([13403, 13404, 13405])

In [50]:
for atom in htf.hybrid_topology.atoms:
    if atom.index in swap_indices:
        print(atom)

HOH3665-O
HOH3665-H1
HOH3665-H2


In [60]:
og_indices = [37523, 37524, 37525]

In [61]:
for og, swap in zip(og_indices, swap_indices):
    print(positions[og])
    print(positions[swap])
    temp = copy.copy(positions[og])
    positions[og] = positions[swap]
    positions[swap] = temp

[1.27297604 6.23424006 3.09856224]
[8.02543449 1.14856029 1.17571199]
[1.20960546 6.30275393 3.07729387]
[7.94678164 1.13592839 1.22878313]
[1.21940422 6.15773773 3.11953354]
[8.0821352  1.20366669 1.22966194]


In [62]:
positions[37523]

array([8.02543449, 1.14856029, 1.17571199])

In [63]:
positions[37524]

array([7.94678164, 1.13592839, 1.22878313])

In [64]:
positions[37525]

array([8.0821352 , 1.20366669, 1.22966194])

In [65]:
positions[13403]

array([1.27297604, 6.23424006, 3.09856224])

In [66]:
positions[13404]

array([1.20960546, 6.30275393, 3.07729387])

In [67]:
positions[13405]

array([1.21940422, 6.15773773, 3.11953354])